In [1]:
# !pip install -U chardet urllib3 gensim

In [2]:
cd ../src

/workspace/kaggle_otto_rs/src


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [5]:
import os
import gc
import sys
import cudf
import json
import glob
import numba
import pickle
import warnings
import itertools
import numpy as np
import pandas as pd

from tqdm import tqdm
from datetime import datetime
from collections import Counter
from numerize.numerize import numerize

from gensim.models import Word2Vec
from gensim.test.utils import common_texts

warnings.simplefilter(action="ignore", category=FutureWarning)

ModuleNotFoundError: No module named 'gensim'

In [ ]:
from params import *

### Params

In [ ]:
MODE = "test"

In [ ]:
if MODE == "val":
    files = glob.glob("../output/full_train_parquet/*") + glob.glob(
        "../output/val_parquet/*"
    )
elif MODE == "test":
    files = glob.glob("../output/full_train_val_parquet/*") + glob.glob(
        "../output/test_parquet/*"
    )
else:
    raise NotImplementedError

In [ ]:
sessions = cudf.concat([cudf.read_parquet(f) for f in files], ignore_index=True)
sessions = sessions.sort_values(['session', 'ts'])
sessions = sessions[['session', 'aid']].groupby(['session']).agg(list)

sentences = sessions['aid'].to_pandas().apply(lambda x: x.tolist()).to_list()

del sessions
gc.collect()
numba.cuda.current_context().deallocations.clear()

In [ ]:
%%time

w2vec = Word2Vec(
    sentences=sentences, 
    vector_size=50, 
    epochs=5, 
    sg=1, 
    window=3, 
    sample=1e-3, 
    ns_exponent=1, 
    min_count=1, 
    workers=8
)

In [ ]:
aid2idx = {aid: i for i, aid in enumerate(w2vec.wv.index_to_key)}

In [ ]:
emb = {}
for aid, idx in aid2idx.items():
    emb[aid] = w2vec.wv.vectors[idx]

In [ ]:
w2vec.wv.vectors.shape

In [ ]:
pickle.dump(
    emb,
    open(f'../output/matrix_factorization/word2vec_{MODE}.emb', 'wb')
)

np.save(f'../output/matrix_factorization/word2vec_{MODE}.npy', w2vec.wv.vectors)

Done !